### Tanzinian Water Pump Classification ###

### Data Cleaning

This notebook serves a purpose to show data cleaning for the training labels and values data. 
Here we will deal with null values, missing values, and outliers. 

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pickle 
import numpy as np

pd.set_option('precision', 4)
pd.options.display.max_seq_items = None


### Read in training labels and values: 

In [3]:
Y = pd.read_csv('DATA/TRAINING_LABELS.csv')
df = pd.read_csv('DATA/TRAINING_VALUES.csv', index_col = 0)

In [4]:
# Merge Y to the data frame after we looked at values below

df = df.merge(Y, on = 'id')

In [5]:
df.shape

(59400, 41)

First, we look at the training values to assess the data we have and to get a feel for it. 

In [187]:
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.9381,-9.8563,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.6988,-2.1475,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.4607,-3.8213,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.4862,-11.1553,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.1308,-1.8254,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [188]:
df.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.0000,59400.0000,59400.0000,59400.0000,5.9400e+04,59400.0000,59400.0000,59400.0000,59400.0000,59400.0000
mean,37115.1318,317.6504,668.2972,34.0774,-5.7060e+00,0.4741,15.2970,5.6297,179.9100,1300.6525
std,21453.1284,2997.5746,693.1164,6.5674,2.9460e+00,12.2362,17.5874,9.6336,471.4822,951.6205
min,0.0000,0.0000,-90.0000,0.0000,-1.1649e+01,0.0000,1.0000,0.0000,0.0000,0.0000
25%,18519.7500,0.0000,0.0000,33.0903,-8.5406e+00,0.0000,5.0000,2.0000,0.0000,0.0000
50%,37061.5000,0.0000,369.0000,34.9087,-5.0216e+00,0.0000,12.0000,3.0000,25.0000,1986.0000
75%,55656.5000,20.0000,1319.2500,37.1784,-3.3262e+00,0.0000,17.0000,5.0000,215.0000,2004.0000
max,74247.0000,350000.0000,2770.0000,40.3452,-2.0000e-08,1776.0000,99.0000,80.0000,30500.0000,2013.0000


In [189]:
df.shape

(59400, 41)

### Assessing our Target ###
Looking at the values below, we will want to handle this class imbalance later. 


In [190]:
Y.status_group.value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

### Dealing with NaN and Null Values; Checking for Duplicates

We have a lot of null values: funder, installer, subvillage, public_meeting, scheme_management,
    scheme_name, and permit. 


In [191]:
df.isnull().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

Let's deal with the null values first. 

### Funder

Fill in missing or null values with 'Unknown'

In [192]:
#df.funder.value_counts()

In [193]:
#df.installer.value_counts()

In [194]:
df['funder'] = df.funder.fillna('Unknown')
df['installer'] = df.installer.fillna('Unknown')

### Subvillage

In [195]:
#df.subvillage.value_counts()

In [196]:
#df.subvillage.isnull().sum()

In [197]:
freq_subvil = df.groupby(['region']).subvillage.apply(lambda x: x.value_counts().index[0])

In [198]:
df['subvillage'] = np.where(df.subvillage.isnull(), 
                            freq_subvil[df.region], 
                            df.subvillage)

### Public Meeting
Given True is the majority value for public meetings, we want to fill in True. 

In [199]:
#df.public_meeting.value_counts()

In [200]:
df['public_meeting'] = df.public_meeting.fillna(True)

### Scheme Management and Scheme Name 
Fill in the na values Unknown for managment and None for name. 

In [201]:
#df.scheme_management.value_counts()

In [202]:
#df.scheme_name.value_counts()

In [203]:
df['scheme_management'] = df.scheme_management.fillna('Unknown')
df['scheme_name'] = df.scheme_name.fillna('None')

### Permit

If highly class imbalanced, impute the majority, if not assign random bool. 

In [204]:
#df.permit.value_counts()

In [205]:
rand_choice = np.random.choice([True, False], df.permit.isnull().sum())

In [206]:
df['permit']= df.permit.mask(df.permit.isnull(), np.random.choice([True, False], size=len(df)))

In [207]:
df['permit'] = df.permit.astype('bool')

### Double Check for Nulls and Duplicates

In [208]:
df.isnull().sum()

id                       0
amount_tsh               0
date_recorded            0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
subvillage               0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
recorded_by              0
scheme_management        0
scheme_name              0
permit                   0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
s

In [209]:
df[df.duplicated('id')]

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group


 Null values have been removed. No duplicates were found. 
 While we don't have null values now, many of our values are not correct. 

### Data Abnormalities and Outliers


Several of our features seem to be a little off on the numbers. For construction year, these values have a lot of zero values This may take additional research so that imputed values have significance. 

In [210]:
df.columns

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group', 'status_group'],
      dtype='object')

In [211]:
### Amount TSH 

In [212]:
#df['construction_year'].value_counts()

In [213]:
#df['date_recorded'].value_counts()

In [214]:
#list(df[df['construction_year'] == 0].date_recorded)

Of the construction year, we are missing nearly 35% of the data. 
This is a lot of information to drop or to inpute with an average value. 
We decided to keep this data as 0 and keep it in mind for consideration during feature engineering and selection.


In [215]:
len(df[df.construction_year == 0])/len(df)

0.34863636363636363

### Latitude and Longitude 


In [216]:
tmp = df.copy()
tmp = df[df.longitude > 5]
avg_lat_long = tmp.groupby('region')['latitude', 'longitude'].mean()

/Users/dolcikey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [217]:
df['latitude'] = np.where(df.longitude < 5, 
         avg_lat_long['latitude'][df.region], df.latitude)
df['longitude'] = np.where(df.longitude < 5, 
         avg_lat_long['longitude'][df.region], df.longitude)

### GPS Height

Seems that there's another grossly missing number in gps height. 

In [218]:
len(df[df.gps_height == 0]) /len(df)


0.3440740740740741

Many of the populations are listed as 0 which may mean that the wells are not located in populated areas and that people may have to travel from far away to get to them. 

In [219]:
df['population'].value_counts()

0       21381
1        7025
200      1940
150      1892
250      1681
        ...  
3241        1
1960        1
1685        1
2248        1
1439        1
Name: population, Length: 1049, dtype: int64

In [220]:
len(df[df.population == 0]) /len(df)

0.35994949494949496

### Funder


Many of these are misspelled or not combined well such as Babtest, Babtist, Baptist Church etc. 
Going through and correcting as many as possible. 

In [231]:
# Make All Funder Lowercase 
df['funder'] = df['funder'].str.lower()

In [232]:
swedish = ['swedish', 'sweeden', 'sweden', 'people of sweden']
roman_catholic = ['rc/mission', 'rc', 'roman church', 'rc mission', 'rc ch', 'hw/rc', 'cg/rc',
                  'rc missionary', 'world vision/rc church', 'rc msufi', 'cefa/rcchurch', 'rcchurch/cefa', 'rc missi',
                  'rc church/centr', 'rc mofu', 'rc cathoric', 'rc njoro', 'roman cathoric church', 'rc mi',
                  'rc/dwe', 'rc church', 'rc churc', 'r.c', 'rc .church', 'rc c', 'rc church brother',
                  'rc church/cefa', 'rc church/central gover', 'rc mis']
unicef = ['unicrf','uniseg','unicef/cspd','unicet','uniceg','unicef/central', 'unicef', 'unicef/ csp',
          'unicef/african muslim agency', 'unice','unice/ cspd']
netherland = ['nerthlands', 'netherland', 'nethalan', 'holland', 'netherlands', 'natherland', 'holand']
lutheran = ['lutheran church', 'luthe', 'lutheran']
world_bank = ['world bank', 'world bank/government', 'bank', 'w.b', 'wo', 'word', 'word bank',
             'world banks', 'world nk', 'would bank']
world_vision = ['wvt', 'wvc', 'world visiin', 'world vission', 
                'government /world vision', 'world vision', 'world vision/ kkkt',
                'worldvision', 'world vision/adra', 'word divisio', 'world division', ]
private_individual = ['private individual', 'private', 'mwingereza', "rashid seng'ombe", 'doctor mwambi']
ADB = ['afdp', 'adb', 'lga and adb', 'african development bank', 'african development foundation']
baptist = ['babtest', 'babtist', 'buptist', 'batist church', 'baptist church']
unknown = ['0', 'none', 'unknown', 'o', '-', '_', 'not known', 'not kno', 'unknown installer']

In [233]:
cond = [df.funder.str.contains('mganga'), 
        df.funder.str.contains('mwin'), 
        df.funder.isin(['mwanza', 'mwanga town water authority']),
        (df.funder.isin(roman_catholic)) | df.funder.str.contains('roma'), 
        df.funder.isin(unicef), 
        df.funder.isin(netherland), 
        (df.funder.str.contains('kkkt')) | (df.funder.str.contains('elc')) | (df.funder.isin(lutheran)), 
        df.funder.str.contains('danid'), 
        df.funder.str.contains('hes'),
        df.funder.isin(world_bank) | (df.funder.str.contains('world') & df.funder.str.contains('bank')),
        (df.funder.isin(world_vision)) | (df.funder.str.contains('world') & df.funder.str.contains('vision')),
        (df.funder.str.contains('tasa') | (df.funder.str.contains('tass'))),
        df.funder.str.contains('germa'), 
        df.funder.str.contains('distri'), 
        (df.funder.str.contains('dhv')) | (df.funder == 'dh') , 
        df.funder.isin(private_individual), 
        df.funder.str.contains('dws'), 
        df.funder.str.contains('nora'), 
        df.funder.str.contains('tcrs'), 
        df.funder.str.contains('heal'), 
        df.funder.str.contains('dwe'), 
        df.funder.isin(ADB), 
        (df.funder.str.contains('lga')) | (df.funder.str.contains('loca')),
        df.funder.str.contains('amre'), 
        df.funder.str.contains('oxf'), 
        (df.funder.str.contains('fin')) & (df.funder.str.contains('w')), 
        (df.funder.str.contains('jap')) | (df.funder.isin(['jica', 'jaica'])),
        df.funder.str.contains('isf') | (df.funder == 'is'),
        (df.funder.str.contains('chri')) | (df.funder.str.contains('cris')),
        df.funder.str.contains('das'),
        df.funder.str.contains('taca'), 
        df.funder.str.contains('compas'),
        df.funder.str.contains('vil'),
        df.funder.str.contains('conce'),
        df.funder.str.contains('egy'),
        df.funder.str.contains('meth'), 
        df.funder.str.contains('edk'),
        df.funder.str.contains('finl'),
        df.funder.str.contains('irev'),
        df.funder.isin(baptist),
        (df.funder.str.contains('chur'))| df.funder.str.contains('miss'),
        df.funder.isin(unknown),
        df.funder.str.contains('schoo'),
        (df.funder.str.contains('rws')) | (df.funder.str.contains('rural') & df.funder.str.contains('wat')),
        df.funder.str.contains('ded'),
        df.funder.str.contains('oik'),
        (df.funder.str.contains('kil') & df.funder.str.contains('wat')), 
        df.funder.str.contains('comm'), 
        df.funder.str.contains('farm'),
        df.funder.str.contains('apm'),
        df.funder.str.contains('africar'), 
        df.funder.isin(swedish),
        df.funder.str.contains('wfp'), 
        (df.funder.str.contains('wat') & df.funder.str.contains('aid')), 
        df.funder.str.contains('drdp'), 
        (df.funder.str.contains('wat') & df.funder.str.contains('use')), 
        df.funder.str.contains('muni')
       ]
vals = ['mganga', 'mwinjuma_mzee', 'mwanza', 'roman_catholic', 'unicef', 'netherland', 'kkkt', 
       'danida', 'hesawa', 'world_bank', 'world_vision', 'tasaf', 'germany', 'district council', 'dhv', 
        'individual', 'dwsp', 'norad', 'tcrs','ministry_of_health', 'dwe', 'adb', 'lga', 'amref', 'oxfam', 
        'finwater', 'japan', 'isf', 'christian','dasp', 'tacare', 'compassion', 'village', 'concern',
        'egype', 'methodist', 'friedkin', 'finland','irevea', 'baptist','other_church', 'unknown', 'school',
        'rwssp', 'ded', 'oikos', 'killi_water', 'community', 'farm_afr', 'apm', 'africare','sweden', 'wfp', 
        'wateraid', 'drdp', 'wateruser', 'municipal_council'
       ]
df.funder = np.select(cond, vals, df.funder)

In [234]:
# limiting to values with at least 100 observations
other_funders = [x for x in set(df.funder) if len(df[df.funder == x]) < 50]
df['funder'] = np.where(df.funder.isin(other_funders), 'others', df.funder)

### Installers

In [235]:
typos = {'babtest': 'baptist', 'batist': 'baptist', 'batist': 'baptist', 'belgiam': 'belgium', 
        'cathoric': 'catholic', 'cebtral': 'central', 'cetral': 'central', 'christan': 'christian', 
        'comunity': 'community', 'compasion': 'compassion', 'cristan': 'christian', 'danda': 'danida', 
         'daniad': 'danida', 'danids': 'danida', 'denish': 'danish', 'embasy': 'embassy', 'goldstar': 'gold star',
         'greineker': 'greinaker', 'grobal': 'global', 'gurumeti': 'grumeti', 'holand': 'netherland', 
         'holland': 'netherland','jeica': 'jica', 'jaica': 'jica', 'jicks': 'jica', 'jika': 'jica', 'jiks': 'jica',
         'jsica': 'jica', 'kiliflora': 'killflora', 'kili': 'killi', 'kill': 'killi', 'kuweit': 'kuwait', 
         'lolmoloki': 'lomoloki', 'muslin': 'muslim', 'agrican': 'african', 'cebtral': 'central', 'cental': 'central',
         'cetral': 'central', 'insititutiona': 'institutional', 'morrov': 'morov', 'nampapanga': 'nampopanga', 'nanra': 'nandra',
         'nasan': 'nassan', 'tanload': 'tanroad', 
        }
str_isin = {'swedish': swedish, 'roman_catholic': roman_catholic, 'unicef': unicef, 'netherland': netherland, 
           'lutheran': lutheran, 'world_bank': world_bank, 'world_vision': world_vision, 
           'private': private_individual, 'adb': ADB, 'baptist': baptist, 'unknown': unknown}
str_startswith = {'abd': 'abd', 'adp': 'adp', 'africa m': 'african_muslin', 'african m': 'african_muslim', 
               'africaone': 'africaone', 'aic': 'aic', 'amp': 'amp', 'ang': 'anglican', 'arab': 'arab', 
                 'ardhi': 'ardhi', 'atlas': 'atlas', 'babu': 'babu', 'bingo': 'bingo', 'boni': 'bonite_bottles',
                 'brit': 'britain', 'building': 'building_works', 'calt': 'caltas', 'care': 'care', 
                 'carta': 'cartas', 'ccp': 'ccp', 'centr': 'central_government', 'cg': 'central_government',
                 'chacha': 'chacha', 'chin': 'china', 'christ': 'christian', 'chur': 'church', 'cip': 'cipro', 
                 'cjej': 'cjejow', 'conce': 'concern', 'compa': 'company', 
                 'cons': 'consulting_engineer', 'cosmo': 'cosmos', 'counc': 'council', 'dads': 'dads', 
                 'danid': 'danida', 'dawa': 'dawasa', 'das': 'dasp', 'dbs': 'dbsp', 'ddca': 'ddca', 
                 'desk': 'desk_a_c', 'dh': 'dhv', 'dm': 'dmdd', 'dr': 'dr_matobola', 
                 'dsp': 'dsp', 'dw': 'dwe', 'efa': 'efarm', 'elc': 'kkkt', 'emanda': 'emanda', 
                 'eny': 'enyuati','fram': 'farm_africa', 'fin ': 'fin_water', 'finland': 'gov_finland', 
                  'fin': 'finwater' , 'fp': 'fpct', 'gachu': 'gachuma', 'george': 'george_mtoto', 
                 'german': 'german', 'global':'global_resource', 'gove': 'government', 'grume': 'grumeti',
                 'gwas': 'gwasco', 'hal': 'halmashauri', 'handeni': 'handeni', 'hanja': 'hanja', 'hapa': 'hapa', 
                 'hemed': 'hemed_abdallah', 'hes': 'hesawa', 'hosp': 'hospital', 'hotel': 'hotel_lodge', 
                 'howard': 'howard_humfreys', 'humfrey': 'howard_humfreys', 'icf': 'icf', 'idara': 'idara', 
                 'indiv': 'private', 'ind': 'india', 'insti': 'institutional', 'isf': 'isf', 'islam': 'islamic', 
                 'ital': 'italy', 'jica': 'jica', 'jandu': 'jandu_plumber','jeshi': 'jeshi','juin': 'juin', 
                 'jum': 'juma', 'kaem': 'kaemp', 'kalit': 'kalitasi', 'kanisa': 'kanisa', 'karumba': 'karumba_building', 
                 'kibo': 'kibo', 'killflora': 'killflora', 'killi': 'killiwater', 'kk': 'kkkt', 'kyasha': 'kyasha',
                 'lawate': 'lawate_fuka', 'lind': 'lindi', 'lion': 'lions', 'livi': 'living_water', 'local': 'locals', 
                 'losa': 'losa-kia', 'luthe': 'lutheran', 'mack': 'macdonald', 'maga': 'magadini', 'maji': 'maji', 
                 'makon': 'makonde', 'masjid': 'masjid', 'maswi': 'maswi','missi': 'missionary', 'oik': 'oikos',
                 'mkon': 'mkongo', 'mombo': 'mombo', 'active tank': 'active_tank', 'besad': 'besada', 'mdrd': 'mdrdp',
                  'mile': 'millennium', 'mlaki': 'mlaki', 'morovi' : 'morovian', 'msiki': 'msikitini', 'muslim': 'muslims',
                  'mwaki': 'mwaki', 'mwalimu': 'mwalimu', 'mwita': 'mwita', 'naishu': 'naishu', 'nandra': 'nandra',
                  'ngo': 'ngo', 'nora': 'norad', 'noshad': 'noshad', 'nyiak': 'nyiakaliangani', 'pente': 'pentecost', 
                  'peter':'individual', 'pet': 'pet_corp', 'plan': 'plan', 'pnr': 'pnr', 'priv': 'private', 
                  'qu' : 'quick_win', 'qw': 'quick_win', 'regi': 'regional_water', 'regwa': 'regwa', 
                  'resolute': 'resolute', 'rhobi': 'rhobi', 'roma': 'roman_catholic', 'rotar': 'rotary_club',
                  'rude': 'rudep', 'rural': 'rural_water_supply', 'rw': 'rwe', 'samwel': 'samweli', 'save': 'save_the_rain', 
                  'saxon': 'saxon', 'schoo': 'school', 'sda': 'sda', 'secon': 'secondary_school', 
                  'soli': 'solidame', 'tanap': 'tanapa', 'tanz': 'tanzania', 'tasa': 'tasaf', 'tass': 'tassaf', 
                  'tcrs': 'tcrs', 'tlc': 'tlc', 'total': 'total_landcare', 'toto': 'total_landcare', 'tpp': 'tpp', 
                  'tscr': 'tcrs', 'tsrc': 'tcrs', 'tukwa': 'tukwale', 'u.s.a': 'usa', 'ubalozi': 'ubalozi_wa_marekani', 
                  'umoja': 'umoja', 'us embassy': 'usa', 'usa embassy': 'usa', 'vicf': 'vicf', 'victoria': 'victoria', 
                  'vifa': 'vifaf', 'vitecos': 'vitecos', 'wamis': 'missionary', 'wasso': 'wasso', 'water use': 'water_user_group', 
                  'water': 'wateraid', 'wed': 'wedeco', 'winam': 'winam', 'winnin': 'winnin', 'wizara': 'wizara', 
                  'ww': 'wwf', 'yasini':'yasini', 'zao': 'zao_water'
                 }
str_contains = {'adr':'adra', 'amre':'amre', 'man dr': 'aquaman_drilling', 'baptist': 'baptist', 
               'japan': 'japan', 'munic': 'municipal', 'commu': 'community', 'vil': 'village', 
               'lwi': 'lwi','distri': 'district', 'of water': 'ministry of water',  'egypt': 'egypt',}
str_endswith = {'isan': 'artisan'}

In [236]:
for k, v in typos.items():
    df.installer = df.installer.apply(lambda x: x.replace(k, v))
for k, v in str_isin.items():
    df.installer = np.where(df.installer.isin(v), k, df.installer)
for k, v in str_startswith.items():
    df.installer = np.where(df.installer.str.startswith(k), v, df.installer)
for k, v in str_contains.items():
    df.installer = np.where(df.installer.str.contains(k), v, df.installer)
for k, v in str_endswith.items():
    df.installer = np.where(df.installer.str.endswith(k), v, df.installer)


In [237]:
# limiting to values with at least 100 observations
other_installer = [x for x in set(df.installer) if len(df[df.installer == x]) < 100]
df['installer'] = np.where(df.installer.isin(other_installer), 'others', df.installer)

In [238]:
df.to_pickle('clean_df.pkl')